# SABR
#### Roland Grinis - MIPT & GrinisRIT

In [1]:
import torch
from torch.utils.cpp_extension import load
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline
torch.manual_seed(987654);

In [2]:
!mkdir -p build

In [4]:
sabr = load(name='sabr',
             build_directory='./build',
             sources=['sabr.cc'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=True)

The input conditions for extension module sabr have changed. Bumping to version 1 and re-building as sabr_v1...
Emitting ninja build file ./build/build.ninja...
Building extension module sabr_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/2] c++ -MMD -MF sabr.o.d -DTORCH_EXTENSION_NAME=sabr_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1014\" -isystem /home/rolandgrinis.linux/miniconda3/envs/noa/lib/python3.9/site-packages/torch/include -isystem /home/rolandgrinis.linux/miniconda3/envs/noa/lib/python3.9/site-packages/torch/include/torch/csrc/api/include -isystem /home/rolandgrinis.linux/miniconda3/envs/noa/lib/python3.9/site-packages/torch/include/TH -isystem /home/rolandgrinis.linux/miniconda3/envs/noa/lib/python3.9/site-packages/torch/include/THC -isystem /home/rolandgrinis.linux/miniconda3/envs/noa/include/python3.9 -D_GLIB

In [6]:
a = torch.randn(5, dtype=torch.float64)

In [9]:
sabr.test(torch.randn(5))

tensor([-0.4044,  0.6905,  1.3833,  0.5268,  0.7891])

## 1. Implied Volatility

Zelen & Severo approximation for the standard normal CDF $\text{N}(x) = \frac{1}{\sqrt{2\pi}}\int^x_{-\infty} \text{d}z \exp(-z^2/2)$ is given by: 
$$
\text{N}(x) = 1 - \text{N}'(x) \sum_{i=1}^{5} b_i t^i + \epsilon (x)
$$
where $t = (1 + b_0 x)^{-1}$, the error is bounded by $|\epsilon (x)| < 7.5 \cdot 10^{-8}$ and the coefficients $b_i$ are fixed (see implementation).


### References
* Hagan, P. S. et al. *Implied volatility formulas for Heston models.* Wilmott 2018.98 (2018)
* Bartlett, B. *Hedging Under SABR Model.* Wilmott Magazine, (2006)
* Oblój, J. *Fine-Tune Your Smile: Correction to Hagan et al*. preprint arXiv:0708.0998 (2007)
* Paulot, L. *Asymptotic implied volatility at the second order with application to the SABR model.* In Large Deviations and Asymptotic Methods in Finance (pp. 37-69). Springer, Cham. (2015)
* Henry-Labordère, P. *A general asymptotic implied volatility for stochastic volatility
models.* http://arxiv.org/abs/cond-mat/0504317 (2005)